# Update Protein Identifiers to UniProt

Note: Requires internet connection to download information from the UniProt.

## Setup
### Import packages

In [1]:
import numpy as np
import pandas as pd
from rbc_gem_utils import (
    build_string,
    check_database_release_online,
    get_dirpath,
    show_versions,
)
from rbc_gem_utils.database.uniprot import (
    UNIPROT_DB_TAG,
    UNIPROT_ID_RE,
    get_release_UniProt,
    query_UniProt,
)

# Display versions of last time notebook ran and worked
show_versions()


Package Information
-------------------
rbc-gem-utils 0.0.3

Dependency Information
----------------------
beautifulsoup4                       4.13.4
bio                                   1.8.0
cobra                                0.29.1
depinfo                               2.2.0
gurobipy                             12.0.3
matplotlib                           3.10.3
matplotlib-venn                       1.1.2
memote                               0.17.0
networkx                                3.5
notebook                              7.4.4
openpyxl                              3.1.5
pandas                                2.3.1
pre-commit                            4.2.0
rbc-gem-utils[database,network,vis] missing
requests                             2.32.4
scikit-learn                          1.7.0
scipy                                1.16.0
seaborn                              0.13.2

Build Tools Information
-----------------------
pip          25.1
setuptools 78.1.1
wheel      0.45

## Check UniProt release
If the release does not match the expected release, it is because database has been updated since the last time this code was utilized. 

* According to [UniProt](https://www.uniprot.org/help/downloads), updates to the database are made every eight weeks.
* If the current release does not match the expected release, it is because database has been updated since the last time this code was utilized.
    * If the notebook works without needing any significant modifications, the only update needed is to the release in the [uniprot.py](../../src/rbc_gem_utils/database/uniprot.py) source code file to resolve the issue.

In [2]:
release = get_release_UniProt()

use_interim = not check_database_release_online(UNIPROT_DB_TAG, verbose=True, **{})
# Use different directory paths for unexpected behavior
if use_interim:
    warn(
        "Online release of database has been updated since the last time notebook was used."
    )


database_dirpath = get_dirpath(
    "database", UNIPROT_DB_TAG, use_temp="interim" if use_interim else None
)
annotation_dirpath = get_dirpath(
    "annotation", use_temp="interim" if use_interim else None
)

# Ensure directories exist
database_dirpath.mkdir(exist_ok=True, parents=True)
annotation_dirpath.mkdir(exist_ok=True, parents=True)

Current and expected releases match. Current release: 2025_03


## Load aggregated proteomic data, raw

In [3]:
overwrite = True

excel_filepath_raw = (
    get_dirpath("proteomics", use_temp="external") / "proteomics_aggregated_raw.xlsx"
)

### Load obsolete identifier mapping
Meant to get all IDs into UniProt identifiers. Does not account for obselete/deleted/old UniProt IDs. Those must be manually checked

In [4]:
df_obsolete = pd.read_csv(
    get_dirpath("proteomics", use_temp="external") / "proteomics_obsolete.tsv",
    sep="\t",
    dtype=str,
    index_col=None,
)
df_obsolete

,UniProt,IPI,GI,EntryNames,UniParc,Other,Known issue?
0,A0A024QZ64,NaN,78070601,NaN,UPI0000000C53,NaN,NaN
1,A0A024R0L6,NaN,4505587;14043830,NaN,UPI0000131074,NaN,NaN
2,A0A024R1A3,NaN,23510338,NaN,UPI0000137946,NaN,NaN
3,A0A024R1I3,NaN,10092677;40674427,NaN,UPI000006D362,NaN,NaN
4,A0A024R4T4,NaN,4507791,NaN,UPI0000020ECC,NaN,NaN
...,...,...,...,...,...,...,...
3559,E9PM46,NaN,NaN,NaN,NaN,NaN,Merged with other UniProt
3560,P27824,NaN,NaN,NaN,NaN,NaN,Merged with other UniProt
3561,Q9BQA1,NaN,NaN,NaN,NaN,NaN,Merged with other UniProt
3562,Q9P035,NaN,NaN,NaN,NaN,NaN,Merged with other UniProt


### Load table of contents, raw

In [5]:
df_contents_raw = pd.read_excel(
    excel_filepath_raw, sheet_name="Table of Contents", dtype=str
).fillna("")
df_contents_raw

,PubMed/Sheet Name,ID type,Year,Publication,Notes
0,12362340,UniProt,2002,"Low TY, Seow TK, Chung MC. Separation of human...",Used for iAB-RBC-283
1,14963112,GI,2004,"Kakhniashvili DG, Bulla LA Jr, Goodman SR. The...",
2,16861337,IPI,2006,"Pasini EM, Kirkegaard M, Mortensen P, Lutz HU,...",Used for iAB-RBC-283
3,18399644,IPI,2008,"Simó C, Bachi A, Cattaneo A, Guerrier L, Forti...",
4,18494517,IPI,2008,"Ringrose JH, van Solinge WW, Mohammed S, O'Fla...",
5,18614565,IPI,2008,"Roux-Dalvai F, Gonzalez de Peredo A, Simó C, G...",Used for iAB-RBC-283
6,19778645,GI,2009,"van Gestel RA, van Solinge WW, van der Toorn H...",
7,22157974,GI,2011,"D'Amici GM, Rinalducci S, Zolla L. Depletion o...",
8,22954596,UniProt,2012,"Pesciotta EN, Sriswasdi S, Tang HY, Mason PJ, ...",Disease conditions
9,23781972,GI,2013,"Pallotta V, D'Alessandro A, Rinalducci S, Zoll...",


### Map other identifiers to UniProt

In [6]:
updated_table_dict = {"Table of Contents": df_contents_raw.copy()}
obsolete_dict = {}
problems = {}
index_name = "Uniprot"
for idx, (sheet_name, id_type) in df_contents_raw[
    ["PubMed/Sheet Name", "ID type"]
].iterrows():
    df = pd.read_excel(
        excel_filepath_raw, sheet_name=sheet_name, usecols=[0], dtype=str
    )
    df.columns = [id_type]
    if id_type != "UniProt":
        # Map to obsolete identifiers
        df_obsolete_mapping = (
            df_obsolete[[id_type, "UniProt"]].dropna(how="all").drop_duplicates().copy()
        )
        if id_type == "GI":
            df[id_type] = df[id_type].apply(
                lambda x: x.split("|")[-1] if str(x).startswith("gi|") else x
            )
        df_obsolete_mapping = df.merge(
            df_obsolete_mapping, left_on=id_type, right_on=id_type, how="left"
        )
        obsolete_dict[sheet_name] = df_obsolete_mapping.copy()
        counts = df_obsolete_mapping.nunique()
        counts["Obsolete"] = (
            df_obsolete_mapping["UniProt"].isna().value_counts()[True].item()
        )

        counts = {k: v for (k, v) in sorted(counts.to_dict().items())}
        updated_table_dict["Table of Contents"].loc[idx, "ID type"] = "UniProt"
        note_str = build_string([f"{v} {k}" for k, v in counts.items() if v != 0])
        updated_table_dict["Table of Contents"].loc[idx, "Notes"] = note_str
        updated_table_dict[sheet_name] = list(
            df_obsolete_mapping["UniProt"].dropna().unique()
        )
    else:
        check_mixed_ids = df["UniProt"][
            ~df["UniProt"].apply(lambda x: True if UNIPROT_ID_RE.search(x) else False)
        ]
        counts = {"UniProt": 0}
        if not check_mixed_ids.empty:
            problems[sheet_name] = df
        df["UniProt"] = df["UniProt"].apply(
            lambda x: (
                UNIPROT_ID_RE.search(x).group()
                if isinstance(x, str) and UNIPROT_ID_RE.search(x)
                else x
            )
        )
        counts.update({"UniProt": df["UniProt"].nunique()})
        counts = {k: v for (k, v) in sorted(counts.items())}
        note_str = build_string([f"{v} {k}" for k, v in counts.items() if v != 0])
        updated_table_dict["Table of Contents"].loc[idx, "Notes"] = note_str
        updated_table_dict[sheet_name] = list(df["UniProt"].dropna().unique())

updated_table_dict["Table of Contents"]

,PubMed/Sheet Name,ID type,Year,Publication,Notes
0,12362340,UniProt,2002,"Low TY, Seow TK, Chung MC. Separation of human...",84 UniProt
1,14963112,UniProt,2004,"Kakhniashvili DG, Bulla LA Jr, Goodman SR. The...",183 GI;128 Obsolete;66 UniProt
2,16861337,UniProt,2006,"Pasini EM, Kirkegaard M, Mortensen P, Lutz HU,...",590 IPI;197 Obsolete;393 UniProt
3,18399644,UniProt,2008,"Simó C, Bachi A, Cattaneo A, Guerrier L, Forti...",153 IPI;42 Obsolete;111 UniProt
4,18494517,UniProt,2008,"Ringrose JH, van Solinge WW, Mohammed S, O'Fla...",699 IPI;310 Obsolete;389 UniProt
5,18614565,UniProt,2008,"Roux-Dalvai F, Gonzalez de Peredo A, Simó C, G...",1577 IPI;364 Obsolete;1214 UniProt
6,19778645,UniProt,2009,"van Gestel RA, van Solinge WW, van der Toorn H...",561 GI;180 Obsolete;426 UniProt
7,22157974,UniProt,2011,"D'Amici GM, Rinalducci S, Zolla L. Depletion o...",134 GI;214 Obsolete;44 UniProt
8,22954596,UniProt,2012,"Pesciotta EN, Sriswasdi S, Tang HY, Mason PJ, ...",831 UniProt
9,23781972,UniProt,2013,"Pallotta V, D'Alessandro A, Rinalducci S, Zoll...",140 GI;47 Obsolete;118 UniProt


### Map UniProt IDs to current UniProt

In [7]:
all_failed = {}
all_unmapped = {}

In [8]:
# Extract all relevant information for now and save
query_parameters = {
    "query": " && ".join(
        [
            "(organism_id:9606)",  # Homo sapiens (Human)
        ]
    ),
    "format": "tsv",
    "size": 500,
    "compressed": True,
    "fields": ",".join(
        [
            "reviewed",
            "accession",
            "gene_primary",
        ]
    ),
}

idx = -1
# Use redo sheets to remap specific sheets that failed
redo_sheets = set([])
issues = ['28689405']
for sheet_name, query_ids in updated_table_dict.copy().items():
    if sheet_name == "Table of Contents" or (
        redo_sheets and sheet_name not in redo_sheets
    ):
        idx += 1
        continue
    print(f"{idx}) {sheet_name}\n{(4 + len(sheet_name)) * '-'}")
    try:
        df_results, uniparc, failed_ids, unmapped_ids = query_UniProt(
            query_ids,
            query_parameters=query_parameters,
            from_db="UniProtKB",
            to_db="UniProtKB",
            return_failed=True,
        )
    except:
        # Try additional time before continuing
        print(f"Issue with {sheet_name}, retry after extraction is finished\n")
        idx += 1
        issues += [sheet_name]
        continue
    if failed_ids:
        print(f"Failed IDS: {failed_ids}\n")
        all_failed[sheet_name] = set(failed_ids)
    if unmapped_ids:
        print(f"Unmmaped IDS: {unmapped_ids}\n")
        all_unmapped[sheet_name] = set(unmapped_ids)
    df_uniprot = df_results["Entry"].drop_duplicates().dropna().reset_index(drop=True)
    df_uniprot.name = index_name
    updated_table_dict[sheet_name] = df_uniprot
    updated_table_dict["Table of Contents"].loc[
        idx, "Notes"
    ] += f" -->  {len(df_uniprot)} UniProt {release}"
    idx += 1
    print()
print(issues)
updated_table_dict["Table of Contents"]

0) 12362340
------------


Number of failed query IDs : 5


Fetched: 79 / 79


Number of failed IDs : 3


Failed IDS: {'182074', '7710117', '4503581'}

Unmmaped IDS: {'Q64689', 'P80315'}


1) 14963112
------------
Fetched: 66 / 66

2) 16861337
------------
Fetched: 396 / 396

3) 18399644
------------
Fetched: 112 / 112

4) 18494517
------------
Fetched: 389 / 389

5) 18614565
------------
Fetched: 500 / 1217
Fetched: 1000 / 1217
Fetched: 1217 / 1217

6) 19778645
------------
Fetched: 427 / 427

7) 22157974
------------
Fetched: 44 / 44

8) 22954596
------------
Fetched: 500 / 765


...

Number of failed query IDs : 2


Fetched: 1812 / 1812


Number of obsolete IDs : 2


Unmmaped IDS: {'P0DN79', 'A0A0B4J2D5'}


29) 38147558
------------
Fetched: 500 / 1208
Fetched: 1000 / 1208
Fetched: 1208 / 1208

30) 38964323
------------
Fetched: 500 / 2601
Fetched: 1000 / 2601
Fetched: 1500 / 2601
Fetched: 2000 / 2601
Fetched: 2500 / 2601
Fetched: 2601 / 2601

31) 40043591
------------
Fetched: 346 / 346

['28689405']


,PubMed/Sheet Name,ID type,Year,Publication,Notes
0,12362340,UniProt,2002,"Low TY, Seow TK, Chung MC. Separation of human...",84 UniProt --> 79 UniProt 2025_03
1,14963112,UniProt,2004,"Kakhniashvili DG, Bulla LA Jr, Goodman SR. The...",183 GI;128 Obsolete;66 UniProt --> 66 UniProt...
2,16861337,UniProt,2006,"Pasini EM, Kirkegaard M, Mortensen P, Lutz HU,...",590 IPI;197 Obsolete;393 UniProt --> 396 UniP...
3,18399644,UniProt,2008,"Simó C, Bachi A, Cattaneo A, Guerrier L, Forti...",153 IPI;42 Obsolete;111 UniProt --> 112 UniPr...
4,18494517,UniProt,2008,"Ringrose JH, van Solinge WW, Mohammed S, O'Fla...",699 IPI;310 Obsolete;389 UniProt --> 389 UniP...
5,18614565,UniProt,2008,"Roux-Dalvai F, Gonzalez de Peredo A, Simó C, G...",1577 IPI;364 Obsolete;1214 UniProt --> 1217 U...
6,19778645,UniProt,2009,"van Gestel RA, van Solinge WW, van der Toorn H...",561 GI;180 Obsolete;426 UniProt --> 427 UniPr...
7,22157974,UniProt,2011,"D'Amici GM, Rinalducci S, Zolla L. Depletion o...",134 GI;214 Obsolete;44 UniProt --> 44 UniProt...
8,22954596,UniProt,2012,"Pesciotta EN, Sriswasdi S, Tang HY, Mason PJ, ...",831 UniProt --> 765 UniProt 2025_03
9,23781972,UniProt,2013,"Pallotta V, D'Alessandro A, Rinalducci S, Zoll...",140 GI;47 Obsolete;118 UniProt --> 119 UniPro...


#### Quick check unmapped against known deleted entries

In [9]:
known_issues = df_obsolete[["UniProt", "Known issue?"]].dropna(subset="Known issue?")
known_issues = known_issues.set_index("UniProt")["Known issue?"].to_dict()

for k, values in all_unmapped.items():
    print(f"Sheet: {k}")
    for v in values:
        if not UNIPROT_ID_RE.search(v):
            # print(f"{v}\tNot a UniProt ID")
            continue

        elif v in known_issues:
            # print(f"{v}\t{known_issues[v]}")
            continue
        else:
            print(f"{v}\tUnclear")
    print()
    print()

Sheet: 12362340


Sheet: 22954596
P35241	Unclear


Sheet: 24555563


Sheet: 26078478


Sheet: 26271157


Sheet: 26474164
P62937	Unclear
P31939	Unclear
P11586	Unclear


Sheet: 27006477
P63010	Unclear
O14744	Unclear
P78371	Unclear
P06748	Unclear
P62495	Unclear


Sheet: 27777340
Q8WZ42	Unclear


Sheet: 28263177
P01772	Unclear


Sheet: 28689405


Sheet: 30271928
P13746	Unclear


Sheet: 33341364


Sheet: 33676898


Sheet: 37942280




### Export aggregated proteomic data, updated IDs


In [10]:
if overwrite:
    with pd.ExcelWriter(
        get_dirpath("proteomics", use_temp="external") / "proteomics_aggregated.xlsx"
    ) as writer:
        for sheet_name, df in updated_table_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)